<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [1]:
import json
import pandas as pd
import tweepy

with open('..\credentials.json') as f:
    keys = json.load(f)

# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [10]:
tweets = tweepy.Cursor(api.search,
                       q=['paro nacional -RT'],
                       since='2021-05-19',
                       until='2021-05-20', 
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(2000)

In [11]:
# Objeto generador
t = [tweet for tweet in tweets]
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    t_processed.append(temp)
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,"Jóvenes y comunidad del municipio de #Ubaté, C...",2021-05-19 23:59:32,Bogotá,"Ubaté, Colombia",1395167267449708544,https://twitter.com/i/web/status/1395167267449...
1,Jajajaja dizque “El congreso tumba reforma a l...,2021-05-19 23:59:07,Bogotá,"Bogotá, D.C., Colombia",1395167163749634053,https://twitter.com/i/web/status/1395167163749...
2,"Hoy en #ZonaFranca, hablamos con un joven que ...",2021-05-19 23:57:34,Bogotá,"Bogotá, D.C., Colombia",1395166772492439552,https://twitter.com/i/web/status/1395166772492...
3,🚨 Investigan muerte en Buenaventura durante di...,2021-05-19 23:57:00,Bogotá,"Bogotá, Colombia",1395166633237389315,https://twitter.com/i/web/status/1395166633237...
4,Viva el paro nacional .... \n\n#vivaelparo\n#V...,2021-05-19 23:55:31,Bogotá,"Bogotá, D.C., Colombia",1395166257767452672,https://twitter.com/i/web/status/1395166257767...


In [12]:
import re 

data = data[['tweets', 'date']]

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

In [13]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,texto_limpio,links,hashtags,mentions,emojis
0,"Jóvenes y comunidad del municipio de #Ubaté, C...",2021-05-19 23:59:32,Jóvenes y comunidad del municipio de Cundinam...,"[https://t.co/SJysHw8xYe, https://t.co/v7ATCuo...","[#Ubaté,, #ParoNacional19M, #ParoNacional]",[],[]
1,Jajajaja dizque “El congreso tumba reforma a l...,2021-05-19 23:59:07,Jajajaja dizque “El congreso tumba reforma a l...,[],"[#ReformaALaSalud, #ParoNacional19M]",[],"[🤡, 🤡]"
2,"Hoy en #ZonaFranca, hablamos con un joven que ...",2021-05-19 23:57:34,Hoy en hablamos con un joven que ha estado en...,[https://t.co/iKHmHy3ED8],"[#ZonaFranca,]","[@nicoarroyavena, @cancinodiegoa, @GloriaDiazM]",[]
3,🚨 Investigan muerte en Buenaventura durante di...,2021-05-19 23:57:00,🚨 Investigan muerte en Buenaventura durante di...,"[https://t.co/QdR5AUsO3q, https://t.co/y3EEF4B...",[],[],"[🚨, ➡️]"
4,Viva el paro nacional .... \n\n#vivaelparo\n#V...,2021-05-19 23:55:31,Viva el paro nacional .... \n\n\n,[https://t.co/8GTHeIIwu5],"[#vivaelparo, #VamosColombia]",[],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [18]:
data['hora'] = data['date'].dt.floor('15T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')

In [19]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)


trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por cada 15 minutos (19-Mayo)")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [20]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)

trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)